In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 18.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 9.7 MB/s eta 0:00:00


In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
# Load pre-trained SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
/home/drusniel/llm_notebooks/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/drusniel/llm_notebooks/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


In [8]:
def preprocess_text(text):
    """
    Tokenizes the input text into sentences.
    """
    sentences = text.split('. ')
    return [sentence.strip() for sentence in sentences if sentence]

def get_sentence_embeddings(sentences):
    """
    Converts sentences to embeddings using SentenceTransformer.
    """
    return model.encode(sentences)

def match_sentences(text1, text2, threshold=0.75):
    """
    Matches sentences from text1 to text2 based on cosine similarity of their embeddings.
    """
    sentences1 = preprocess_text(text1)
    sentences2 = preprocess_text(text2)
    
    logger.info("Sentences from text1: %s", sentences1)
    logger.info("Sentences from text2: %s", sentences2)
    
    embeddings1 = get_sentence_embeddings(sentences1)
    embeddings2 = get_sentence_embeddings(sentences2)
    
    similarity_matrix = cosine_similarity(embeddings1, embeddings2)
    logger.info("Similarity matrix: %s", similarity_matrix)
    
    matches = []
    unmatched1 = set(range(len(sentences1)))
    unmatched2 = set(range(len(sentences2)))
    
    for i, row in enumerate(similarity_matrix):
        for j, similarity in enumerate(row):
            if similarity > threshold:
                matches.append((sentences1[i], sentences2[j], similarity))
                unmatched1.discard(i)
                unmatched2.discard(j)
    
    unmatched_sentences1 = [sentences1[i] for i in unmatched1]
    unmatched_sentences2 = [sentences2[i] for i in unmatched2]
    
    return matches, unmatched_sentences1, unmatched_sentences2

def main():
    text1 = ("As the sun sank below the horizon, casting shadows across Thistledown, "
             "a group of adventurers gathered around a flickering campfire. The air was full of the scent of pine and the distant calls of nocturnal animals.\n\n"
             "\"Are you sure this is the right path, Elaria?\" asked Thorne, his hand on the hilt of his sword. He peered out at the darkening forest warily.\n\n"
             "\"The map led us here,\" said Elaria, a slim elf with piercing green eyes. \"The ancient runes spoke of a temple hidden beyond the Silverstream. We must trust in the old ways.\"\n\n"
             "Brakkar, the burly dwarf, sighed and adjusted his axe. \"Trusting in old runes and forgotten temples... This had better lead to treasure worth all this trouble.\"\n\n"
             "\"Not all treasures are made of gold, Brakkar,\" said Lyra, the group's sorceress. Her eyes reflected the light of the fire. \"Some secrets are far more valuable.\"\n\n"
             "Thorne smiled. \"Secrets or gold... We'll find out soon enough.\"")

    text2 = ("The sun was just setting, and Thistledown was darkening by degrees. A campfire burned at the center of a small group of adventurers—some humans, some elves, and one dwarf. The air held the scent of pine, and the distant call of nocturnal animals echoed through the forest.\n\n"
             "\"Are you sure this is right?\" Thorne asked Elaria, who was pointing at a trail. \"We're supposed to be going toward the Silverstream.\"\n\n"
             "\"The map we found led us here,\" said Elaria, an elf. Her green eyes glinted with a hint of mischief in the firelight. \"It spoke of a temple hidden beyond the river. We'll see if those old runes hold any truth.\"\n\n"
             "\"Trusting in old runes and forgotten temples?\" Brakkar said, adjusting his axe. \"Hope it leads to treasure worth all this trouble.\"\n\n"
             "Lyra smiled at that. \"Not all treasures are made of gold. Some secrets are far more valuable.\"")
    
    matches, unmatched1, unmatched2 = match_sentences(text1, text2)
    
    logger.info("Matched sentences:")
    for match in matches:
        logger.info("Text1: %s\nText2: %s\nSimilarity: %.2f", match[0], match[1], match[2])
    
    logger.info("Unmatched sentences in text1: %s", unmatched1)
    logger.info("Unmatched sentences in text2: %s", unmatched2)

if __name__ == "__main__":
    main()

INFO:__main__:Sentences from text1: ['As the sun sank below the horizon, casting shadows across Thistledown, a group of adventurers gathered around a flickering campfire', 'The air was full of the scent of pine and the distant calls of nocturnal animals.\n\n"Are you sure this is the right path, Elaria?" asked Thorne, his hand on the hilt of his sword', 'He peered out at the darkening forest warily.\n\n"The map led us here," said Elaria, a slim elf with piercing green eyes', '"The ancient runes spoke of a temple hidden beyond the Silverstream', 'We must trust in the old ways."\n\nBrakkar, the burly dwarf, sighed and adjusted his axe', '"Trusting in old runes and forgotten temples..', 'This had better lead to treasure worth all this trouble."\n\n"Not all treasures are made of gold, Brakkar," said Lyra, the group\'s sorceress', 'Her eyes reflected the light of the fire', '"Some secrets are far more valuable."\n\nThorne smiled', '"Secrets or gold..', 'We\'ll find out soon enough."']
INFO:_

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:__main__:Similarity matrix: [[ 0.47909266  0.7808743   0.36432648  0.19050679  0.40290454  0.18935221
   0.25128692  0.06008013  0.04477419  0.0572561 ]
 [ 0.34853593  0.3029966   0.87715924  0.49618113  0.25114632  0.35833463
   0.3336891   0.27747256  0.12796298  0.18529359]
 [ 0.3810831   0.22356963  0.5719634   0.729069    0.3822515   0.3341831
   0.32547677  0.3643644   0.04114632  0.14702854]
 [ 0.20595044  0.23514651  0.2690522   0.5077661   0.2075455   0.76063037
   0.65164995  0.3201077   0.42035162  0.2961071 ]
 [ 0.17049411  0.196282    0.26436275  0.28011551  0.24448109  0.18634276
   0.6388257   0.25879085  0.1826383   0.21228862]
 [ 0.12994505  0.19518682  0.20609137  0.33249357  0.09088229  0.54658014
   0.8029504   0.29744738  0.3872131   0.28365856]
 [ 0.01581878  0.14806704  0.19834086  0.3171805   0.1086188   0.2519964
   0.3933123   0.6488729   0.7007144   0.52296954]
 [ 0.35829353  0.3785791   0.4154824   0.21939783  0.7132244   0.24633218
   0.13539533  0.245

The initial split by . might not capture all sentence boundaries properly, especially when sentences end with other punctuation marks or newlines. We can enhance the preprocessing step to handle various sentence boundaries more accurately. Additionally, we'll adjust the similarity threshold

In [9]:
import re
import numpy as np
import logging

def preprocess_text(text):
    """
    Tokenizes the input text into sentences using regular expressions to handle different punctuation.
    """
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence]

def match_sentences(text1, text2, threshold=0.65):
    """
    Matches sentences from text1 to text2 based on cosine similarity of their embeddings.
    """
    sentences1 = preprocess_text(text1)
    sentences2 = preprocess_text(text2)
    
    logger.info("Sentences from text1: %s", sentences1)
    logger.info("Sentences from text2: %s", sentences2)
    
    embeddings1 = model.encode(sentences1)
    embeddings2 = model.encode(sentences2)
    
    similarity_matrix = cosine_similarity(embeddings1, embeddings2)
    logger.info("Similarity matrix: %s", similarity_matrix)
    
    matches = []
    unmatched1 = set(range(len(sentences1)))
    unmatched2 = set(range(len(sentences2)))
    
    for i, row in enumerate(similarity_matrix):
        for j, similarity in enumerate(row):
            if similarity > threshold:
                matches.append((sentences1[i], sentences2[j], similarity))
                unmatched1.discard(i)
                unmatched2.discard(j)
    
    unmatched_sentences1 = [sentences1[i] for i in unmatched1]
    unmatched_sentences2 = [sentences2[i] for i in unmatched2]
    
    return matches, unmatched_sentences1, unmatched_sentences2


In [10]:
text1 = ("As the sun sank below the horizon, casting shadows across Thistledown, "
         "a group of adventurers gathered around a flickering campfire. The air was full of the scent of pine and the distant calls of nocturnal animals.\n\n"
         "\"Are you sure this is the right path, Elaria?\" asked Thorne, his hand on the hilt of his sword. He peered out at the darkening forest warily.\n\n"
         "\"The map led us here,\" said Elaria, a slim elf with piercing green eyes. \"The ancient runes spoke of a temple hidden beyond the Silverstream. We must trust in the old ways.\"\n\n"
         "Brakkar, the burly dwarf, sighed and adjusted his axe. \"Trusting in old runes and forgotten temples... This had better lead to treasure worth all this trouble.\"\n\n"
         "\"Not all treasures are made of gold, Brakkar,\" said Lyra, the group's sorceress. Her eyes reflected the light of the fire. \"Some secrets are far more valuable.\"\n\n"
         "Thorne smiled. \"Secrets or gold... We'll find out soon enough.\"")

text2 = ("The sun was just setting, and Thistledown was darkening by degrees. A campfire burned at the center of a small group of adventurers—some humans, some elves, and one dwarf. The air held the scent of pine, and the distant call of nocturnal animals echoed through the forest.\n\n"
         "\"Are you sure this is right?\" Thorne asked Elaria, who was pointing at a trail. \"We're supposed to be going toward the Silverstream.\"\n\n"
         "\"The map we found led us here,\" said Elaria, an elf. Her green eyes glinted with a hint of mischief in the firelight. \"It spoke of a temple hidden beyond the river. We'll see if those old runes hold any truth.\"\n\n"
         "\"Trusting in old runes and forgotten temples?\" Brakkar said, adjusting his axe. \"Hope it leads to treasure worth all this trouble.\"\n\n"
         "Lyra smiled at that. \"Not all treasures are made of gold. Some secrets are far more valuable.\"")

matches, unmatched1, unmatched2 = match_sentences(text1, text2)

logger.info("Matched sentences:")
for match in matches:
    logger.info("Text1: %s\nText2: %s\nSimilarity: %.2f", match[0], match[1], match[2])

logger.info("Unmatched sentences in text1: %s", unmatched1)
logger.info("Unmatched sentences in text2: %s", unmatched2)


INFO:__main__:Sentences from text1: ['As the sun sank below the horizon, casting shadows across Thistledown, a group of adventurers gathered around a flickering campfire.', 'The air was full of the scent of pine and the distant calls of nocturnal animals.', '"Are you sure this is the right path, Elaria?" asked Thorne, his hand on the hilt of his sword.', 'He peered out at the darkening forest warily.', '"The map led us here," said Elaria, a slim elf with piercing green eyes.', '"The ancient runes spoke of a temple hidden beyond the Silverstream.', 'We must trust in the old ways."\n\nBrakkar, the burly dwarf, sighed and adjusted his axe.', '"Trusting in old runes and forgotten temples...', 'This had better lead to treasure worth all this trouble."\n\n"Not all treasures are made of gold, Brakkar," said Lyra, the group\'s sorceress.', 'Her eyes reflected the light of the fire.', '"Some secrets are far more valuable."\n\nThorne smiled.', '"Secrets or gold...', 'We\'ll find out soon enough.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:__main__:Similarity matrix: [[ 0.47573566  0.7700572   0.3480097   0.19624688  0.18512627  0.41992038
   0.18079787  0.25153345  0.06565206  0.02754611  0.04963072]
 [ 0.25398555  0.24631093  0.93369496  0.23228613  0.23945272  0.20347555
   0.22440684  0.07104941  0.05313875  0.01877338  0.11995322]
 [ 0.20076151  0.18589817  0.15572374  0.779638    0.45299086  0.20595007
   0.27051488  0.4088133   0.36706015  0.13421255  0.15534338]
 [ 0.47023493  0.19025743  0.45202413  0.37833774  0.2856888   0.40729955
   0.31118834  0.19335356  0.16766196 -0.04441359  0.03878757]
 [ 0.20362633  0.16230047  0.21887901  0.52519953  0.77917314  0.3359213
   0.26744074  0.3013966   0.44968525  0.10715409  0.18383856]
 [ 0.1723978   0.20987602  0.14959764  0.27120736  0.49368206  0.20101088
   0.74463195  0.62669706  0.29655957  0.3660362   0.2812647 ]
 [ 0.171269    0.18891406  0.11584059  0.2938447   0.25531062  0.2520981
   0.16000132  0.6375573   0.2568256   0.15945925  0.19283184]
 [ 0.10194